# Bandits and Exploration/Exploitation

> In this post, we will learn simple reinforcement learning problem, the bandits, This is the summary of lecture "Fundamentals of Reinforcement Learning" from Coursera.

- toc: true 
- badges: true
- comments: true
- author: Chanseok Kang
- categories: [Python, Coursera, Reinforcement_Learning]
- image: 

## Sequential Decision Making with Evaluative Feedback

### Action-Values (or Action-Value function)

The value is the expected reward

$$ \begin{aligned} q_{*}(a) &\doteq \mathbb{E}[R_t \vert A_t = a] \quad \forall a \in \{1, \dots, k\} \\ &= \sum_{r} p(r \vert a ) r \end{aligned}$$

Here, $\doteq$ means **"is defined as"**. 

The goal of Action-value is to maximize the expected reward. In formula, we can express it like this,

$$ \arg\max_a q_{*}(a) $$

Then, how can we calculate the $q_{*}(a)$?

In K-armed bandit problem, we can explain it in medicine prescription case. Consider we have three types of medicines.

![medicine](image/bandit_q_a.png)

And each medicine has its own distribution.

![medicine](image/bandit_distribution.png)

Red one is Binomial, green one is normal, and blue one is uniform distribution. Based on this information, we can calculate each $q_{*}(a)$.

$$ \begin{aligned} \text{Red :} q_{*}(a) &= 0.5 \times -11 + 0.5 \times 9 = -0.1 \\ 
   \text{Green :} q_{*}(a) &= 1 \\
   \text{Blue :} q_{*}(a) &= 3 \\ \end{aligned}$$